Bienvenue dans ce second TP d’apprentissage automatique !

La semaine dernière, nous avons utilisé l'apprentissage automatique pour essayer de répondre à la question “Combien d’heures devrais je réviser pour obtenir une bonne note à mon cours d’IA et apprentissage?”.

Cette semaine, nous allons nous intéresser à une question légèrement différente, "Combien d'heures devrais je réviser pour valider mon cours d'IA et apprentissage?".

Pour ce faire, nous allons développer un modèle de régression logistique univariée qui aura pour but d'estimer la probabilité qu'un.e étudiant.e a de valider en fonction du nombre d'heures qu’iel aura consacrées à la révision.

Comme nous l’avons vu précédemment, la première étape nécessaire à l’élaboration de pareil modèle consiste en la récolte et création d’un jeu de données. Pour ce faire, nous avons recueilli l’année dernière des données sur 70 étudiants de la L3 Informatique et vidéoludique de Paris 8. Chaque étudiant a été invité à fournir deux informations cruciales : Le nombre d'heures qu'iel a consacrées à la révision pour le partiel, et la note qu'iel a obtenue à ce partiel. Nous avons minutieusement enregistré ces informations et avons créé un ensemble de données représentatif pour notre analyse.



**Objectif du TP**

Votre mission pour ce TP est de développer un modèle de régression logistique univariée qui permettra d'estimer la probabilité qu'a un.e étudiant.e de valider son partiel en fonction du temps de révision qu'iel consacrera pour le partiel. Vous allez apprendre à utiliser ces données pour construire le modèle, le former et évaluer ses performances.

Commencez par exécutez la case suivante, qui importe toutes les librairies nécessaire à l'execution du TP.

In [ ]:
import torch
import sklearn
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt

#  1. Chargement et visualisation des données

Maintenant que notre environnement de travail est prêt, nous allons pouvoir charger notre jeu de données et regarder un peu à quoi il ressemble. Uploadez le fichier "train.csv" fourni avec ce notebook dans vos fichiers, et chargez le dans le jupyter avec la fonction pandas pd.read_csv

In [ ]:
train_df = # CODEZ ICI #

Affichez une description de votre jeu de données ici en utilisant la méthode .describe() associée à votre dataframe pandas

In [ ]:
# CODEZ ICI #


Ajoutez au DataFrame une variable "a_valide", construite à partir de la variable "note" comme une variable binaire valant 0 (respectivement 1) si l'individu a eu une note strictement inférieure (respectivement supérieure) à 10 sur 20.

In [ ]:
# CODEZ ICI #

En utilisant la librairie pyplot, visualisez la distribution des variables prédictives et à expliquer de votre jeu de données, ainsi que leur relation.

Vous pourrez pour ce faire utiliser les fonctions pyplot plt.hist et plt.scatter

In [ ]:
# CODEZ ICI #

Décrivez ici qualitativement votre jeu de données.

Un modèle de régression logistique semble t'il indiqué ici pour implémenter un algorithme capable de déterminer si un.e étudiant aura validé son partiel à partir de son temps de révision?

(Ecrivez votre réponse dans cette case)

# 2. Création et ajustement d'un modèle de régression linéaire avec Pytorch.

Maintenant que l'on connait mieux notre jeu de données, il est temps d'implémenter et d'ajuster notre modèle. Dans un premier temps, nous utiliserons de nouveau Pytorch.

# 2.1. Implémentation du modèle

Rappelez vous, afin d'implémenter un modèle en Pytorch, il est nécessaire d'implémenter un objet qui hérite de la classe torch.nn.Module. Nous devons en particulier coder les deux méthodes suivantes:



*   La méthode init où l'on définiera les paramètres de notre modèle en tant qu'attributs
*   Une méthode appelée forward, qui admet un argument, input (un tensor Torch) et qui retourne également un Tensor. C'est dans cette méthode que nous implémentererons la logique de notre couche, c'est à dire la transformation que l'on désire que notre modèle applique au Tensor input

Pour rappel, un modèle de régression logistique cherche à estimer la probabilité que la variable à prédire y vaut 1 à partir de la variable d'entrée x en appliquant à cette dernière la transformation suivante:

$$ y = sigmoid(coef * x + ordonnée) $$

PS: La fonction torch.nn.Sigmoid devrait vous être utile pour implémenter votre modèle (attention, comme il est souvent la cas en Pytorch, il s'agit ici aussi également d'un objet)

In [ ]:
class LogisticRegression(torch.nn.Module):
    def __init__(self):
        """
        La méthode __init__ est utilisée pour définir tous les paramètres du
        modèle (les coefficients que l'on va pouvoir faire varier pendant
        la descente de gradient)
        """
        super(LogisticRegression, self).__init__()
        # Définissez ici le coefficient directeur de notre modèle, comme une
        # instance de Parameter initialisé par un Tensor de valeur [1]
        self.coefficient = # CODEZ ICI #
        # Déginissez ici l'ordonnée à l'origine du modèle, comme une
        # instance de Parameter initialisé par un Tensor de valeur [0]
        self.ordonnee = # CODEZ ICI #

    def forward(self, input):
        """
        La méthode forwad est utilisée pour définir la logique de notre modèle.
        c'est à dire la transformation que l'on veut appliquer à l'entrée input
        avec nos paramètres self.coefficient et self.ordonnee
        """
        output = # CODEZ ICI #
        return output

# 2.2. Ajustement du modèle

Comme vu en cours, l'implémentation d'un modèle d'apprentissage machine dépend de la définition de 4 entités distinctes:


*   Un jeu de données constitué d'exemples de paires variable prédictive - variable à prédire
*   Un modèle, définit comme une fonction paramétrique qui prend comme argument la variable prédictive et essaie de rendre en sortie une approximation de la variable à prédire
*   Une fonction objectif (ou fonction coût) qui prend comme argument les paramètre du modèle et rend en sortie un scalaire indiquant à quel point notre modèle "prédit bien" la variable à prédire à partir de la variable prédictive
*   Un algorithme d'optimisation pour trouver le minimum de cette fonction objectif (et ainsi trouver notre modèle comme l'argument de ce minimum)

Le jeu de données étant collecté et le modèle déjà implémenté, il ne nous reste plus qu'à définir notre fonction objectif, et notre algorithme d'optimisation!

Comme on l'a vu la semaine dernière, PyTorch propose tout une variété d'algorithme d'optimisation et de fonction objectif, en fonction des besoins et des envies. Nous nous intérésserons dans ce TP aux deux objets suivant:

*   torch.nn.BCE comme fonction objectif (BCE pour Binary Cross Entropy)
*   torch.optim.SGD comme algorithme de descente de gradient (SGD pour Stochastic Gradient Descent)

In [ ]:
def train_model(X, y, num_iter=10, learning_rate=0.01):

    # On instancie notre modèle
    model = # CODEZ ICI #

    # On définit notre fonction perte
    objectif = # CODEZ ICI #

    # On définit notre algorithme d'optimisation (SGD: Stochastic Gradient Descent)
    optimizer = # CODEZ ICI #

    trajectory = []

    for _ in range(num_iter):
        inputs = torch.tensor(X)
        targets = torch.tensor(y)

        # Calculez les prédictions
        outputs = # CODEZ ICI #

        # Calculer la fonction objectif à partir des prédictions et des targets
        loss = # CODEZ ICI #

        # Calcule le gradient
        # CODEZ ICI#

        # Applique une itération de descente de gradient aux paramètres du modèle
        optimizer.step()

    return model

Maintenant que la fonction train_model est implémentée, nous pouvons l'appeler sur notre jeu de données et obtenir un modèle ajusté!

Pour évaluer ses performances, il nous suffit simplement par la suite d'obtenir les prédictions du modèle en lui donnant en entrée nos variables explicatives, puis de calculer le taux d'exactitude.

Ajuster et évaluer les performances de votre modèle pour les valeurs suivantes de l'argument learning rate: [1, 0.1, 0.01, 0.001]

In [ ]:
X = torch.tensor(train_df.num_heures)
y = torch.tensor(train_df.valide)

for learning_rate in [1., .1, .01, 0.001]:
  # Utilisez la fonction train_model pour ajuster un modèle de régression logistique sur votre jeux de données
  model = # CODEZ ICI #

  # Utilisez le modèle obtenu pour calculez ses prédictions
  predictions = # CODEZ ICI #

  # Calculez le taux d'exactitude du modèle
  accuracy = # CODEZ ICI #

  print("Learning_rate = %.3f : Taux d'exactitude = %.3f" % (learning_rate, accuracy))

# 3. La même chose avec scikit-learn

Félicitation! Vous venez d'implémenter vos premiers modèles de classification avec Pytorch.

Comme nous l'avons vu la semaine dernière, pour implémenter des modèles très simples comme ceux que l'on a vu au cours de ce TP, quelques lignes de codes avec la librairie scikit-learn suffisent.

Voyez plutôt:

In [ ]:
from sklearn.linear_model import LogisticRegression

X = X.numpy().reshape([-1, 1])
y = y.numpy().reshape([-1, 1])

model = LogisticRegression(penalty=None, multiclass='multinomial')
model.fit(X, y)
y_preds = model.predict(X)
accuracy = np.mean(y_preds == y)
print("Accuracy du modèle: %.3f" % accuracy)

# 4. Régression logistique et données déséquilibrées

J'ose espérer que certains d'entre vous, plutôt que de s'intéresser à prédire si vous allez valider ou non votre cours d'IA et apprentissage, préfèreraient savoir s'ils valideront avec une bonne note.

Nous pourrions donc nous intéresser à une question encoree légèrement différente, "Combien d'heures devrais je réviser pour avoir plus de 15 à mon partiel d'IA et apprentissage?"

Ajoutez au DataFrame une variable "a_valide_plus", construite à partir de la variable "note" comme une variable binaire valant 0 (respectivement 1) si l'individu a eu une note strictement inférieure (respectivement supérieure) à 15 sur 20.

En utilisant la librairie pyplot, visualisez la distribution des variables prédictives et à expliquer de votre jeu de données, ainsi que leur relation.

Vous pourrez pour ce faire utiliser les fonctions pyplot plt.hist et plt.scatter

In [ ]:
# CODEZ ICI #

Décrivez ici qualitativement votre jeu de données.

Un modèle de régression logistique semble t'il indiqué ici pour implémenter un algorithme capable de déterminer la note qu'un étudiant aura au partiel à partir de son temps de révision?

Quel choix de métrique proposez vous pour correctement estimer les performances de votre modèle?

(Ecrivez votre réponse dans cette case)

Ajustez un modèle de régression logistique en utilisant la librairie scikit-learn afin de prédire cette nouvelle variable "valide_plus" à partir de la variable "num_heures". Evaluez ensuite ce modèle avec la ou les métrique de votre choix, et calculez également la proportion d'individus que le modèle prédit à 1.

In [ ]:
# CODEZ ICI #